In [1]:
#| default_exp structs

In [2]:
#|export

from flax.struct import dataclass
import jax
import jax.numpy as jnp
from typing import Optional, Tuple, List

@dataclass(frozen=True)
class GeneticMap:
    """
    Defines the genetic map of the organism.

    Attributes:
        chromosome_lengths (jnp.ndarray): The genetic length of each chromosome in Morgans.
            Shape: (n_chr,)
        locus_positions (List[jnp.ndarray]): A list where each element is a JAX array
            containing the Morgan positions of loci for a single chromosome.
    """
    chromosome_lengths: jnp.ndarray
    locus_positions: List[jnp.ndarray]


@dataclass(frozen=True)
class Population:
    """
    Represents a collection of individuals within a breeding program.

    Attributes:
        geno (jnp.ndarray): Genotypes.
            Shape: (n_individuals, n_chr, 2, n_loci)
        ibd (jnp.ndarray): Identity-by-descent tracking for founder alleles.
            Shape: (n_individuals, n_chr, 2, n_loci)
        meta (jnp.ndarray): Metadata [id, mother_id, father_id, birth_gen].
            Shape: (n_individuals, 4)
    """
    geno: jnp.ndarray
    ibd: jnp.ndarray
    meta: jnp.ndarray

@dataclass(frozen=True)
class Trait:
    """
    Defines the genetic architecture of one or more traits.

    This structure holds the information linking genotypes to phenotypes,
    based on an additive QTL model.

    Attributes:
        qtl_chromosome (jnp.ndarray): Chromosome index for each QTL.
            Shape: (n_qtl,)
        qtl_position (jnp.ndarray): Locus index for each QTL within its chromosome.
            Shape: (n_qtl,)
        qtl_effects (jnp.ndarray): The additive effect of each QTL on each trait.
            Shape: (n_qtl, n_traits)
        intercept (jnp.ndarray): The base value for each trait.
            Shape: (n_traits,)
    """
    qtl_chromosome: jnp.ndarray
    qtl_position: jnp.ndarray
    qtl_effects: jnp.ndarray
    intercept: jnp.ndarray

@dataclass(frozen=True)
class BreedingState:
    """
    The complete, self-contained state of a single simulation replicate at a point in time.

    This object is the primary carrier passed between steps in the simulation.

    Attributes:
        population (Population): The current population of individuals.
        key (jax.Array): The JAX pseudo-random number generator key.
        generation (int): The current generation number of the simulation.
    """
    population: Population
    key: jax.Array
    generation: int

In [3]:
#|export

import jax
import jax.numpy as jnp
from typing import Tuple

def quick_haplo(
    key: jax.Array,
    n_ind: int,
    n_chr: int,
    seg_sites: int,
    inbred: bool = False,
    chr_length: float = 1.0
) -> Tuple[Population, GeneticMap]:
    """
    Rapidly simulates founder haplotypes and a corresponding genetic map.

    This function creates a population with random alleles (0/1) at a frequency of 0.5
    and in linkage equilibrium. It also generates a simple, evenly-spaced genetic map.

    Args:
        key: The jax.random.PRNGKey.
        n_ind: Number of individuals.
        n_chr: Number of chromosomes.
        seg_sites: Number of segregating sites per chromosome.
        inbred: If True, individuals will be homozygous.
        chr_length: The genetic length of each chromosome in Morgans. Defaults to 1.0.

    Returns:
        A tuple containing:
          - A `Population` object for the founders.
          - A `GeneticMap` object for the simulation.

    Example:
        >>> import jax
        >>> from chewc.initialization import quick_haplo
        ...
        >>> key = jax.random.PRNGKey(42)
        >>> founder_pop, genetic_map = quick_haplo(
        ...     key=key,
        ...     n_ind=10,
        ...     n_chr=3,
        ...     seg_sites=100
        ... )
        ...
        >>> print(f"Genotype shape: {founder_pop.geno.shape}")
        Genotype shape: (10, 3, 2, 100)
        >>> print(f"Number of chromosomes in map: {len(genetic_map.locus_positions)}")
        Number of chromosomes in map: 3
        >>> print(f"Positions on Chr 1: {genetic_map.locus_positions[0].shape}")
        Positions on Chr 1: (100,)
    """
    # --- Population Generation ---
    if inbred:
        haplotypes = jax.random.randint(key, (n_ind, n_chr, 1, seg_sites), 0, 2, dtype=jnp.int8)
        geno = jnp.broadcast_to(haplotypes, (n_ind, n_chr, 2, seg_sites))
    else:
        geno = jax.random.randint(key, (n_ind, n_chr, 2, seg_sites), 0, 2, dtype=jnp.int8)

    ibd = jnp.broadcast_to(
        jnp.arange(n_ind * 2, dtype=jnp.int32).reshape(n_ind, 1, 2, 1),
        (n_ind, n_chr, 2, seg_sites)
    )

    meta = jnp.stack([
        jnp.arange(n_ind),
        jnp.full((n_ind,), -1),
        jnp.full((n_ind,), -1),
        jnp.zeros(n_ind)],
        axis=-1, dtype=jnp.int32
    )
    population = Population(geno=geno, ibd=ibd, meta=meta)

    # --- Genetic Map Generation ---
    chromosome_lengths = jnp.full((n_chr,), chr_length, dtype=jnp.float32)
    locus_positions = [
        jnp.linspace(0., chr_length, seg_sites) for _ in range(n_chr)
    ]
    genetic_map = GeneticMap(
        chromosome_lengths=chromosome_lengths, locus_positions=locus_positions
    )

    return population, genetic_map

In [4]:
#|export

import jax.numpy as jnp
import jax

@jax.jit
def compute_dosage(population: Population) -> jnp.ndarray:
    """
    Computes the dosage of the '1' allele for each individual at each locus.

    Dosage is calculated as the sum of alleles across the two homologous
    chromosomes, resulting in values of 0, 1, or 2.

    Args:
        population: A `Population` object.

    Returns:
        A 3D array of dosages with shape (n_individuals, n_chr, n_loci).
    """
    # geno shape: (n_individuals, n_chr, 2, n_loci)
    # Sum across the chromosome pair axis (axis=2)
    return jnp.sum(population.geno, axis=2)


def add_trait(
    key: jax.Array,
    founder_pop: Population,
    n_qtl_per_chr: int,
    mean: jnp.ndarray,
    var: jnp.ndarray,
    sigma: Optional[jnp.ndarray] = None,
) -> Trait:
    """
    Creates a Trait architecture and scales its effects to a target
    additive variance in the founder population.

    Args:
        key: The jax.random.PRNGKey for random number generation.
        founder_pop: The founder `Population` object. This is used to
            properly scale the genetic variance.
        n_qtl_per_chr: The number of QTL to sample per chromosome.
        mean: A 1D array for the target mean of each trait. Shape: (n_traits,).
        var: A 1D array for the target additive genetic variance of each trait.
             Shape: (n_traits,).
        sigma: An optional covariance matrix for generating correlated trait effects.
               Shape: (n_traits, n_traits).

    Returns:
        A `Trait` object with QTL effects scaled to the desired parameters.
    """
    key, qtl_key, effect_key = jax.random.split(key, 3)

    n_chr, _, n_loci_per_chr = founder_pop.geno.shape[1:]
    n_total_qtl = n_qtl_per_chr * n_chr

    # 1. Randomly sample QTL locations from all available loci
    all_loci_indices = jnp.arange(n_chr * n_loci_per_chr)
    qtl_loc_flat = jax.random.choice(
        qtl_key, all_loci_indices, (n_total_qtl,), replace=False
    )
    # Convert flat indices to (chromosome, position) pairs
    qtl_chromosome, qtl_position = jnp.divmod(
        jnp.sort(qtl_loc_flat), n_loci_per_chr
    )

    # 2. Generate raw QTL effects
    n_traits = mean.shape[0]
    raw_effects = jax.random.normal(effect_key, (n_total_qtl, n_traits))
    if sigma is not None:
        # Induce correlation between traits using Cholesky decomposition
        cholesky_factor = jnp.linalg.cholesky(sigma)
        effects = raw_effects @ cholesky_factor.T
    else:
        effects = raw_effects

    # 3. Scale effects to match target variance in the founder population
    founder_dosage = compute_dosage(founder_pop)  # Shape: (N, C, L)
    # Get dosage only at the sampled QTL locations
    qtl_dosage = founder_dosage[:, qtl_chromosome, qtl_position]  # Shape: (N, Q)
    
    # Calculate genetic values (gvs) in the founders
    gvs = qtl_dosage @ effects  # Shape: (N, n_traits)

    # Calculate the scaling factor and intercept to match target mean and var
    # Adding a small epsilon to avoid division by zero if founders are identical
    scale = jnp.sqrt(var / (jnp.var(gvs, axis=0) + 1e-8))
    intercept = mean - jnp.mean(gvs, axis=0) * scale
    final_effects = effects * scale

    return Trait(
        qtl_chromosome=qtl_chromosome,
        qtl_position=qtl_position,
        qtl_effects=final_effects,
        intercept=intercept,
    )


In [5]:
key = jax.random.PRNGKey(42)
founder_pop = quick_haplo(
key=key,
n_ind=10,
n_chr=3,
seg_sites=100,
inbred=False
 )

In [6]:
#| hide
import nbdev; nbdev.nbdev_export()

/home/glect/.local/lib/python3.10/site-packages/nbdev/export.py:88: UserWarning: Notebook '/mnt/c/Users/cltng/gdrive/chewc/nbs/03_meiosis.ipynb' uses `#|export` without `#|default_exp` cell.
Note nbdev2 no longer supports nbdev1 syntax. Run `nbdev_migrate` to upgrade.
See https://nbdev.fast.ai/getting_started.html for more information.
  warn(f"Notebook '{nbname}' uses `#|export` without `#|default_exp` cell.\n"
/home/glect/.local/lib/python3.10/site-packages/nbdev/export.py:88: UserWarning: Notebook '/mnt/c/Users/cltng/gdrive/chewc/nbs/05_select.ipynb' uses `#|export` without `#|default_exp` cell.
Note nbdev2 no longer supports nbdev1 syntax. Run `nbdev_migrate` to upgrade.
See https://nbdev.fast.ai/getting_started.html for more information.
  warn(f"Notebook '{nbname}' uses `#|export` without `#|default_exp` cell.\n"
/home/glect/.local/lib/python3.10/site-packages/nbdev/export.py:88: UserWarning: Notebook '/mnt/c/Users/cltng/gdrive/chewc/nbs/06_cross.ipynb' uses `#|export` without `#